This is an Ai assistant designed to help anyone with interview preparation, this will be done in the following steps:
Stage 1: Gathering information via company website
1. Ask the user to input the name of the company and the position.
2. Search through google and confirm with user if the link is the company.
3. When user confirmed the correct company website,

Stage 2 (if the job is listed on company website):
1. Find the active position on website careers page and confirm with user if it's the position that user applied.
1. Using generative feature to combine information from the web scraping and the job title and then write up to 5 typical questions that will be asked in the company industry, Give the user sample answers to each of the questions, and explain the logic reasoning behind each question asked, what employers looking for, what's good answer in this industry etc.

Version 0.1: Build prototype using OpenAI API with beautifulSoup webscraping

In [104]:
import os
import requests
from openai import OpenAI
from bs4 import BeautifulSoup
import json
import gradio as gr
from google.colab import userdata
import markdown

In [105]:
openai_api_key=userdata.get("OPEN_API_KEY")
MODEL="gpt-4o-mini"
openai=OpenAI(api_key=openai_api_key)

In [106]:
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}
class Website:
  def __init__(self, url):
    self.url=url
    response=requests.get(url,headers=headers)
    self.body=response.content
    soup = BeautifulSoup(self.body, 'html.parser')
    self.title=soup.title.string if soup.title else "NO title found"
    if soup.body:
      for irrelavant in soup.body(["script", "style", "img", "input"]):
        irrelavant.decompose()
      self.text=soup.body.get_text(separator="\n", strip=True)
    else:
      self.body=""
    links = [link.get('href') for link in soup.find_all('a')]
    self.links=[link for link in links if link]

  def get_content(self):
    return f"Webpage Title：\n{self.title}\nWebpage Content：\n{self.text}\n\n"

In [107]:
system_prompt = """You are an AI-powered interview preparation assistant. Your primary goal is to help users prepare for job interviews by providing relevant information and practice questions.
You are provided with a list of links found on a company website, and the job title user will be applying for,
generate the top 5 most likely asked question with the job title, and give comprehensive answers with bullet points on the what interviewer is looking for and how to answer them professionally.
Remember to be helpful, informative, and provide the user with valuable insights to help them succeed in their interview.
"""

In [108]:
link_system_prompt:""" You are a helpful assistant who tries to help user on prepare for interviews \
you will be provided with a list of links and decide which links are relevant for this purpose \
and you will decide which links are most relevant, you should respond in JSON as in this example"""

In [109]:
def get_links_user_prompt(website):
  user_prompt=f"Here is the list of links of the website of{website.url}"
  user_prompt+="Please decide which of these are relevant web links for a interview preparation, responde with the full http uRL in JSON format \ Do not include Terms of service, privacy and email links"
  user_prompt+="links(some might be relevant links): \n"
  user_prompt+="\n".join(website.links)
  return user_prompt

In [110]:
def get_links(url):
  website=Website(url)
  response=openai.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": link_system_prompt},
          {"role": "user", "content": get_links_user_prompt(website)},
      ])
  assistant_response=response.choices[0].message.content
  try:
    links=json.loads(assistant_response)
  except json.JSONDecodeError:
    print(f"Warning: Invalid JSON response: {assistant_response}")
    links = {"links": []}  # Return an empty list if JSON decoding fails
  return links


In [111]:
def get_all_details(website_url): # modify this function here
  result="Landing page:\n"
  website = Website(website_url) # assign the website object to a variable
  result+= website.get_content()
  links=get_links(website_url)
  for link in links["links"]:
    result+= f"\n\n{link['type']}\n"
    result+= Website(link["url"]).get_content()
  return result


In [112]:
def prepare_for_interview(website_url, job_title):
    """
    Prepares the user for an interview.

    Args:
        website_url (str): The company's official website URL.
        job_title (str): The job title.

    Returns:
        str: A formatted string containing interview questions, sample answers, and explanations.
    """
    try:

        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": get_all_details(website_url)}, # use website_url as function arguments here
            ]
        )

        assistant_response = response.choices[0].message.content
        return assistant_response
    except requests.exceptions.RequestException as e:
        return f"Error accessing website: {e}"
    except Exception as e:
        return f"An error occurred: {e}"


# Gradio Interface
iface = gr.Interface(
    fn=prepare_for_interview,
    inputs=[
        gr.Textbox(label="Company Website URL"),
        gr.Textbox(label="Job Title"),
    ],
    outputs="text",
    title="AI Interview Preparation Assistant",
    description="Get ready for your interview with personalized practice questions and answers.",
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe3b77a10bcf2c7a30.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
